# Introduction

This Notebook runs for the G2Net Gravitational Wave Detection Kaggle competition started early July 2021.

There are already several interesting public EDAs for the data. Thanks these help me greatly.<BR>
I will be kinda quick on this part here. 

The main focus here is building a TensorFlow pipeline mostly relying on tf.data.Dataset and other build in mechanism preparing provided data in order to feed a Machine Learning model.
 

# TODO
### TO FIX
- Tensorflow io installation facing issues.
- Model  training seems to heavily bottleneck on CPU and would last forever. 
Linked to tfio issue or Dataset handling here? <BR>
Same code @home 4790K/RTX2060 runs 5-15% SSD, 20-30%CPU and 90-100%GPU? To be investigated.

### Spectrogram
- Learning and tuning
- In depth data analysis

### ML Model architecture and tuning 
- First have to fix GPU handling
    
## Latest changes
    - Few typos and code presentation
    - Spectrogram output size lowered to (64, 65)
    - Train dataset spectrogram plotting layout
    - Model fit from loaded dataset
    - Model example simplified and minimum records for fast execution w/o GPU 
    
### Changes history

In [ ]:
# ### 09/07/2021
#    - Saved dataset from tf.data.experimental save method could not be read back correctly.
#     => Fixed by upgrading tensorflow to version 2.5, can now be loaded with the load method. 
#     - Lowered saved dataset records count keeping in 5-6 GB range size for confort. 




In [ ]:
# Upgrade to TensorFlow >= 2.5
!pip uninstall pytorch-lightning -y
!pip install tensorflow -U

# Build Dataset 

## Data presentation

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from matplotlib import pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
# for dirname, _, filenames in os.walk('../input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# import math
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras.layers as Layer
from tensorflow.keras.preprocessing import image


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
print("Tensorflow version: ", tf.__version__)

In [ ]:
# Base repository
BASE_PATH = '../input/g2net-gravitational-wave-detection/'
SPLIT_RATIO = .9
RANDOM_STATE = 42

## Consolidate the data in a pandas DataFrame 

In [ ]:
# 
df = pd.read_csv(os.path.join(BASE_PATH, 'training_labels.csv'))
#sample_submission = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')
df.head()

In [ ]:
def apply_raw_path(row, is_train=True): 
    file_name = row[0]
    if is_train:
        return os.path.join(
            BASE_PATH, 'train',
            file_name[0],
            file_name[1],
            file_name[2],
            file_name + ".npy")
    else:
        return os.path.join(
            BASE_PATH, 'test',
            file_name[0],
            file_name[1],
            file_name[2],
            file_name + ".npy")

df['file_path'] = df.apply(apply_raw_path, args=(True,), axis=1)
df['target'] = df['target'].astype('int8')

In [ ]:
df.head()

Let's have a look at the data

In [ ]:
sample_num = 2579

def plot_samples(sample_list):
    fig,a =  plt.subplots(len(sample_list), 3, figsize=(15,2 * len(sample_list)))
    plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=.5, wspace=0.2)
    cpt = 0
    for sample_num in sample_list:
        sample = np.load(df.loc[sample_num]['file_path'])
        a[cpt, 0].plot(sample[0],color='red')
        a[cpt, 1].plot(sample[1],color='green')
        a[cpt, 2].plot(sample[2],color='cyan')
        a[cpt, 1].set_title(
            'sample ' + str(sample_num) +
            ', label ' + str(df.loc[sample_num]['target']),
            fontsize=16
        )
        cpt = cpt + 1
    plt.show()
    return sample.shape

samples = [0, 1, 2, 2547]
plot_samples(samples)


In [ ]:
print("Sample shape: ",str(np.load(df.loc[2547]['file_path']).shape))
print("Number of records:", str(len(df)))

So now we have acces to the data, that is
- 560 000 records 
- Of shape (3, 4096)
Each record includes 3 channels each containing a wave signal sampled at 2048Hz over 2 seconds.
Each channel comes from either one of the two LIGOs either the VIRGO

Now let's build a TensorFlow DataSet preparing data for futur use.

# TensorFlow Dataset preparation

First thing, dataset are only working sequentially and cannot skip (there is a skip method yet it only discards, not really skip).

Considering the whole data volumetry:
- If any full shuffling required it has to be performed on the ds_train dataframe as prior,
- If any validation data to be reserved, rather prepare a spedcific dataset from a dedicated dataframe.

Therefore we will first shuffle the df dataframe then split in train and validation.  

In [ ]:
df.head()

### First shuffle and split

In [ ]:
df_train, df_valid = train_test_split(
    df,
    test_size=1-SPLIT_RATIO,
    train_size=SPLIT_RATIO,
    random_state=RANDOM_STATE)
print(len(df_train))
print(len(df_valid))
display(df_train.head())
print(df_train.loc[0])

### Second step the numpy format

As per my understanding TF does not provide yet an easy way feeding from numpy file type.
Still we can work our way out handling known data structure from files leveraging:
tf.data.FixedLengthRecordDataset

This will require providing with the specs of the data.

The below function found somewhere on the Internet will parse the numpy header returning its length.<BR>
Btw can be found at https://stackoverflow.com/questions/48889482/feeding-npy-numpy-files-into-tensorflow-data-pipeline

In [ ]:
def npy_header_offset(npy_path):
    with open(str(npy_path), 'rb') as f:
        if f.read(6) != b'\x93NUMPY':
            raise ValueError('Invalid NPY file.')
        version_major, version_minor = f.read(2)
        if version_major == 1:
            header_len_size = 2
        elif version_major == 2:
            header_len_size = 4
        else:
            raise ValueError('Unknown NPY file version {}.{}.'.format(version_major, version_minor))
        header_len = sum(b << (8 * i) for i, b in enumerate(f.read(header_len_size)))
        header = f.read(header_len)
        if not header.endswith(b'\n'):
            raise ValueError('Invalid NPY file.')
        return f.tell()

In [ ]:
# records chosen in train df indexes / raise error if was allocated to test df
for record_num in [0, 1, 3, 25, 2586, 54863, 258412, 559998]:
    file_length = os.path.getsize(df_train.loc[record_num]['file_path'])
    header_size = npy_header_offset(df_train.loc[record_num]['file_path'])
    print(
        "File length:", file_length,
        ", Header size: ",  header_size,
        ", Data length", file_length - header_size
    )

Good let's rock
we need: 
- FixedLengthRecordDataset will read the file (can leverage threading btw preventing from cpu/io bottleneck)
- Next tf.io.decode_raw will unflat the data to our expected format (3, 4096)
Still 3 x 4096 != 98304, in fact 98304 / ( 3x4096) = 8 ;<BR> Guess we are facing float 64 data type assuming file type is byte.


In [ ]:
ds_train_data = tf.data.FixedLengthRecordDataset(
    df_train['file_path'],
    98304,
    header_bytes=128,
    num_parallel_reads=4)
ds_train_data = ds_train_data.map(lambda s: tf.reshape(tf.io.decode_raw(s, tf.float64), (3,4096)))
ds_train_data = ds_train_data.map(lambda s: tf.cast(s, tf.float32))

Let's add the labels to our data the zip way

In [ ]:
ds_train_label = tf.data.Dataset.from_tensor_slices(df_train['target'])
ds_train = tf.data.Dataset.zip((ds_train_data, ds_train_label)) 

Does it work?<BR>
Let's adapt previous plot function to work with dataset data:

In [ ]:
def plot_from_dataset(data, label):
    fig,a =  plt.subplots(1, 3, figsize=(15,2))
    plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=.5, wspace=0.2)
    a[0].plot(data[0],color='red')
    a[1].plot(data[1],color='green')
    a[2].plot(data[2],color='cyan')
    a[1].set_title(', label ' + str(label), fontsize=16)
    plt.show()
    return data.shape

In [ ]:
for data, label in ds_train.take(3):
    plot_from_dataset(data.numpy(), label.numpy())

In [ ]:
df_train.index[100]

Good looks pretty solid,
Let's check number wise:

Note: relying on df_train.index matching the right element from dataset hence the:<BR>
    df_train.loc[df_train.index[100]]

In [ ]:
check_game_num = 15
for data, label in ds_train.skip(check_game_num).take(1):
    data_from_ds = data.numpy()
    print(data_from_ds)
    print("label:", label.numpy() )

data_from_file = np.load(
    df_train.loc[df_train.index[check_game_num]]['file_path']).astype(np.float32)
print(data_from_file)
print("label:", df_train.loc[df_train.index[check_game_num]]['target'])

print("")
if (data_from_ds==data_from_file).all():
    print("Hello World!")
else:
    print("Something went wrong.")

# Dataset transformation (Spectrogram)

To sum up we now have at disposal a shuffled train TensorFlow Dataset able to read data from provided files.
- Data have shape (3,4096) float 32 type
- Labels have shape (1) type int8

Now imagine we'd like to leverage a convolutional neural network could be a ResNet, EfficientNet, or other famous ones expecting 2 dimensional image like inputs in RGB mode.
This because I saw in other notebooks here some spectrograms transformation sounds quite relevant for the classification task we target.

I will below propose an implementation idea I have no clue whether it's relevant or not.<BR>
Worth mentioning here I'm new to Machine Learning and worse in regards to signal analysis and tranformation, so please forgive the chosen values if aberrant.
    
So my basic idea is why not transforming each wave into a spectrogram and leveraging one of the 3 RGB channel per wave. 
3 RGB channel xpected, 3 waves at disposal... KIS
    
The data shape tranformation mechanic is the focus here.    


In [ ]:
# We start from: 
print(ds_train.cardinality)
print(ds_train_data.cardinality)

In [ ]:
# lets leverage the tfio.audio TensorFlow tfio library
!pip install  tensorflow_io
import tensorflow_io as tfio


In [ ]:
ds_train_data = tf.data.FixedLengthRecordDataset(
    df_train['file_path'],
    98304,
    header_bytes=128,
    num_parallel_reads=4)
ds_train_data = ds_train_data.map(
    lambda s: tf.reshape(
        tf.io.decode_raw(
            s, tf.float64), (3,4096)))
ds_train_data = ds_train_data.map(
    lambda s: tf.cast(s, tf.float32))
ds_train_data = ds_train_data.map(
    lambda s: tfio.audio.spectrogram(
        s,
        nfft=128,
        window=256,
        stride=64))
print(ds_train_data.cardinality)

# The nfft and stride selected here allow for output size from (4096) to (256,129)
# My guess is the transformation should consider both expected relevant signal transformation and sizing complying with models 

In [ ]:
for data in ds_train_data.take(1):
    print(data)

In [ ]:
# tfio.audio also provides with a melscale method might be leveraged

In [ ]:
# The below applies a log transformation
# the main goal is transforming inputs to a [0,1] compliant with expected inputs hence the shift and scale
# The clip_by_value prevents from log(0) nan.
# Obviously the clip values will have to be set from consistant data analysis # TODO #  

In [ ]:
ds_train_data = ds_train_data.map(
    lambda s: (
        tf.math.log(
            tf.clip_by_value(s,1e-30, 1e-17)) + 60)/25)
print(ds_train_data.cardinality)

In [ ]:
# this has yet to be transposed for each wave becoming one of the RGB channels expected last by TensorFlow
ds_train_rgb = ds_train_data.map(lambda s: tf.transpose(s))
print(ds_train_rgb.cardinality)

# include labels in dataset the zip way
ds_train = tf.data.Dataset.zip((ds_train_rgb, ds_train_label))

### Let's try plotting this

In [ ]:
fig,a =  plt.subplots(2,5, figsize=(18,8))
cpt = 0
min_value = 1
max_value = 0
for data, label in ds_train.skip(15).take(10):
#     data = tf.math.log(data)
    local_max = np.amax(data.numpy())
    if  local_max > max_value:
        max_value = local_max 
    local_min = np.amin(data.numpy())
    if  local_min < min_value:
        min_value = local_min 
#     print(data)
#     a.imshow(tf.math.log(data[0]))
    line = cpt % 5
    col = cpt // 5
    a[col, line].imshow(data)
    a[col, line].set_title(
            'Label: ' + str(label.numpy()),
            fontsize=16)
    cpt = cpt + 1

plt.show()
print("Min value: ", min_value)
print("Max value: ", max_value)


### Build validation dataset 

In [ ]:
# Build validation
ds_valid_label = tf.data.Dataset.from_tensor_slices(df_valid['target'])
ds_valid_data = tf.data.FixedLengthRecordDataset(
    df_valid['file_path'],
    98304,
    header_bytes=128,
    num_parallel_reads=4)
ds_valid_data = ds_valid_data.map(
    lambda s: tf.reshape(tf.io.decode_raw(s, tf.float64), (3,4096)))
ds_valid_data = ds_valid_data.map(
    lambda s: tf.cast(s, tf.float32))
ds_valid_data = ds_valid_data.map(
    lambda s: tfio.audio.spectrogram(
        s,
        nfft=128,
        window=256,
        stride=64))
ds_valid_data = ds_valid_data.map(
    lambda s: (tf.math.log(tf.clip_by_value(s,1e-30, 1e-17)) + 60)/25)
ds_valid_rgb = ds_valid_data.map(
    lambda s: tf.transpose(s))
ds_valid = tf.data.Dataset.zip((ds_valid_rgb, ds_valid_label))

In [ ]:
print(ds_train.cardinality)
print(ds_valid.cardinality)

# Dataset save and load
This is informative and should not be necessary yet might help identifying model fitting performances issues 

In [ ]:
# Save  
tf.data.experimental.save(ds_train.take(120000), './ds_train', compression=None) 
tf.data.experimental.save(ds_valid.take(12000), './ds_valid', compression=None) 


In [ ]:
# Load 
train = tf.data.experimental.load('./ds_train')
valid = tf.data.experimental.load('./ds_valid')

# Fit a RESNET 50 

In [ ]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input

def prep(img, label):
    ret = preprocess_input(img)
    return ret,label

In [ ]:
# Limiting to 2000 record for fast run without CPU
BATCH_SIZE = 16

train_data = train.take(1000).map(prep).batch(BATCH_SIZE).prefetch(buffer_size=64)
valid_data = valid.take(100).map(prep).cache().batch(BATCH_SIZE).prefetch(buffer_size=64)

In [ ]:
base_model = ResNet50V2(
    weights='imagenet',
    include_top=False,
    input_shape=(65,64,3),
    pooling='avg')
base_model.trainable=True
# base_model.trainable=False


fine_tune_at = 171
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

In [ ]:
metrics = [tf.keras.metrics.AUC(name='AUC'),
           tf.keras.metrics.SparseCategoricalAccuracy(name='Accuracy'),
          ]

tb_callback = tf.keras.callbacks.TensorBoard("logs")

callbacks= [
    tb_callback,
]

def create_model(alpha):
    model = tf.keras.Sequential([
        Layer.InputLayer(input_shape=(65,64,3)),
        base_model,
        Layer.Dropout(.3),
        Layer.LeakyReLU(alpha=alpha),
        Layer.Dense(64),
        Layer.Dropout(.3),
        Layer.LeakyReLU(alpha=alpha),
        Layer.Dense(8),
        Layer.Dropout(.3),
        Layer.LeakyReLU(alpha=alpha),
        Layer.Dense(1, activation='sigmoid'),
    ])
    return model


In [ ]:
model = create_model(0)
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=1e-5,
    ),
    loss='binary_crossentropy', metrics=metrics)

In [ ]:
model.fit(
    train_data,
    epochs=2,
    validation_data=valid_data,
    callbacks = callbacks,
    )